In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy import data, datasets

import spacy
import numpy as np

import random
import math
import time

In [2]:
# 각 언어에 맞는 tokenizer 불러오기 

spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [3]:
def tokenize_de(text):
    # 독일어 tokenize해서 단어들을 리스트로 만든 후 reverse 
    # return [tok.text for tok in spacy_de.tokenizer(text)][::-1]
    return [tok.text for tok in spacy_de.tokenizer(text)]
    
def tokenize_en(text):
    # 영어 tokenize해서 단어들을 리스트로 만들기
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [4]:
a = 'two young , white males are outside near many bushes.'
tokenize_en(a)

['two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.']

In [5]:
# SRC = source = input 
#SRC = data.Field(tokenize = tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True,batch_first = True)
SRC = data.Field(tokenize = tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)

# TRG = target = output
#TRG = data.Field(tokenize = tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True,batch_first = True)
TRG = data.Field(tokenize = tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)


In [6]:
# exts : 어떤 언어 사용할지 명시 (input 언어를 먼저 씀)
# fields = (입력, 출력) 
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))

In [7]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [8]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [9]:
print(SRC.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x00000299B36BD610>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '.': 4, 'ein': 5, 'einem': 6, 'in': 7, 'eine': 8, ',': 9, 'und': 10, 'mit': 11, 'auf': 12, 'mann': 13, 'einer': 14, 'der': 15, 'frau': 16, 'die': 17, 'zwei': 18, 'einen': 19, 'im': 20, 'an': 21, 'von': 22, 'sich': 23, 'dem': 24, 'mädchen': 25, 'junge': 26, 'vor': 27, 'zu': 28, 'steht': 29, 'männer': 30, 'sitzt': 31, 'hund': 32, 'den': 33, 'straße': 34, 'während': 35, 'gruppe': 36, 'hält': 37, 'spielt': 38, 'das': 39, 'hemd': 40, 'personen': 41, 'über': 42, 'drei': 43, 'eines': 44, 'frauen': 45, 'blauen': 46, 'neben': 47, 'ist': 48, 'kind': 49, 'roten': 50, 'weißen': 51, 'stehen': 52, 'sitzen': 53, 'menschen': 54, 'am': 55, 'aus': 56, 'spielen': 57, 'durch': 58, 'bei': 59, 'geht': 60, 'trägt': 61, 'fährt': 62, 'wasser': 63, 'um': 64, 'kinder': 65, 'kleines': 66, 'person': 67, 'macht': 68, 'springt': 69, 'kleiner': 70, 'schwa

In [10]:
print(TRG.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x00000299CDA20970>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, 'a': 4, '.': 5, 'in': 6, 'the': 7, 'on': 8, 'man': 9, 'is': 10, 'and': 11, 'of': 12, 'with': 13, 'woman': 14, ',': 15, 'two': 16, 'are': 17, 'to': 18, 'people': 19, 'at': 20, 'an': 21, 'wearing': 22, 'shirt': 23, 'young': 24, 'white': 25, 'black': 26, 'his': 27, 'while': 28, 'blue': 29, 'men': 30, 'red': 31, 'sitting': 32, 'girl': 33, 'boy': 34, 'dog': 35, 'standing': 36, 'playing': 37, 'group': 38, 'street': 39, 'down': 40, 'walking': 41, '-': 42, 'front': 43, 'her': 44, 'holding': 45, 'one': 46, 'water': 47, 'three': 48, 'by': 49, 'women': 50, 'up': 51, 'green': 52, 'little': 53, 'for': 54, 'child': 55, 'looking': 56, 'outside': 57, 'as': 58, 'large': 59, 'through': 60, 'brown': 61, 'yellow': 62, 'children': 63, 'person': 64, 'from': 65, 'their': 66, 'hat': 67, 'ball': 68, 'into': 69, 'small': 70, 'next': 71, 'other': 72,

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [12]:
print(device)
print(torch.cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 3080


In [13]:
batch_size = 128
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=batch_size)

In [14]:
batch = next(iter(train_iter)) # 첫번째 미니배치


In [15]:
batch.src.shape

torch.Size([28, 128])

In [16]:
batch.src

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [3556,    8,    5,  ...,   18,    5,   17],
        [ 640,   26,   13,  ...,   41,   13,   16],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])

In [17]:
batch.trg

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,    4,  ...,   16,    4,    7],
        [3989,   24,    9,  ...,   19,    9,  120],
        ...,
        [   5,    1,    1,  ...,    1,    1,    1],
        [   3,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, embed_dim)
        #self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout, batch_first= True)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src len, batch_size]
        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        
        # outputs = [src len, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        return hidden, cell

In [19]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch size]

        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        # 따라서 hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        # input = [1, batch size]
        input = input.unsqueeze(0)
        
        # embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
               
        # output = [seq len =1 , batch size, hid dim ]
        # hidden = [n layers , batch size, hid dim]
        # cell = [n layers , batch size, hid dim]



        # Decoder에서 항상 seq len = 1
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        # prediction = [batch size, output dim]
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell
        

In [20]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
      
       
    def forward(self, src, trg):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
   
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # output을 저장할 tensor를 만듭니다.(처음에는 전부 0으로)
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # src문장을 encoder에 넣은 후 hidden, cell값을 구합니다.
        
        hidden, cell = self.encoder(src)
        
        # decoder에 입력할 첫번째 input입니다.
        # 첫번째 input은 모두 <sos> token입니다.
        # trg[0,:].shape = BATCH_SIZE 
        input = trg[0,:]  
        
        
        # 한번에 batch_size만큼의 token들을 독립적으로 계산
        # 즉, 총 trg_len번의 for문이 돌아가며 이 for문이 다 돌아가야지만 하나의 문장이 decoding됨
        # 또한, 1번의 for문당 128개의 문장의 각 token들이 다같이 decoding되는 것'''
        for t in range(1, trg_len):
            
            # input token embedding과 이전 hidden/cell state를 decoder에 입력합니다.
            # 새로운 hidden/cell states와 예측 output값이 출력됩니다.
            output, hidden, cell = self.decoder(input, hidden, cell)

            #output = [batch size, output dim]

            # 각각의 출력값을 outputs tensor에 저장합니다.
            outputs[t] = output

            # predictions들 중에 가장 잘 예측된 token을 top에 넣습니다.
            # 1차원 중 가장 큰 값만을 top1에 저장하므로 1차원은 사라집니다.
            # top1 = output.argmax(1) 
            # top1 = [batch size]            
            input = trg[t]
        
        return outputs

In [21]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
EMB_DIM = 256
EMB_DIM = 256

HID_DIM = 512
N_LAYERS = 2

DROPOUT = 0.5

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)


model = Seq2Seq(enc, dec, device).to(device)

In [22]:
optimizer = optim.Adam(model.parameters())

# <pad> 토큰의 index를 넘겨 받으면 오차 계산하지 않고 ignore하기
# <pad> = padding
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = trg_pad_idx)

In [23]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss=0
    
    for i, batch in enumerate(iterator):
        src = batch.src.to(device)
        trg = batch.trg.to(device)

        optimizer.zero_grad()
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        # loss 함수는 2d input으로만 계산 가능 
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len-1) * batch size]
        # output = [(trg len-1) * batch size, output dim)]
        loss = criterion(output, trg)
        
        loss.backward()
              
        optimizer.step()
        
        epoch_loss+=loss.item()
        
    return epoch_loss/len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src.to(device)
            trg = batch.trg.to(device)
            
            output = model(src, trg)
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)
            
            epoch_loss+=loss.item()
        
        return epoch_loss/len(iterator)
            

In [30]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    
    train_loss = train(model, train_iter, optimizer, criterion)
    valid_loss = evaluate(model, valid_iter, criterion)     
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2model.pt')


    print("[epoch: %d] val loss:%.2f | val PPL :%.2f" % (epoch, train_loss, math.exp(train_loss)))
    print("[epoch: %d] val loss:%.2f | val PPL :%.2f" % (epoch, valid_loss, math.exp(valid_loss)))



[epoch: 0] val loss:3.21 | val PPL :24.88
[epoch: 0] val loss:2.99 | val PPL :19.96
[epoch: 1] val loss:3.03 | val PPL :20.68
[epoch: 1] val loss:2.88 | val PPL :17.74
[epoch: 2] val loss:2.88 | val PPL :17.80
[epoch: 2] val loss:2.75 | val PPL :15.64
[epoch: 3] val loss:2.75 | val PPL :15.59
[epoch: 3] val loss:2.67 | val PPL :14.41
[epoch: 4] val loss:2.63 | val PPL :13.90
[epoch: 4] val loss:2.61 | val PPL :13.57
[epoch: 5] val loss:2.53 | val PPL :12.54
[epoch: 5] val loss:2.54 | val PPL :12.67
[epoch: 6] val loss:2.44 | val PPL :11.42
[epoch: 6] val loss:2.49 | val PPL :12.00
[epoch: 7] val loss:2.35 | val PPL :10.50
[epoch: 7] val loss:2.46 | val PPL :11.65
[epoch: 8] val loss:2.27 | val PPL :9.71
[epoch: 8] val loss:2.43 | val PPL :11.31
[epoch: 9] val loss:2.20 | val PPL :9.00
[epoch: 9] val loss:2.40 | val PPL :11.01


In [31]:
model.load_state_dict(torch.load('seq2model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f} |')

| Test Loss: 2.394 | Test PPL: 10.961 |
